In [ ]:
import requests
import pandas as pd
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as bs
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.tokenize import RegexpTokenizer
import os

In [ ]:
# Function for Syllable Counting
def syllable_counter(word):
    word = word.lower()
    count = 0
    vowels = 'aeiouy'
    for i  in range(0,len(word)):
        if word[i] in vowels and word[i - 1] not in vowels:
            count += 1
    if word.endswith('es') or word.endswith('ed'):
        count -= 1
    if count< 0:
        count == 0
    return count

In [ ]:
#Funtion for counting Complex words
def complex_word(word):
    op = syllable_counter(word)
    if op > 2:
        return op
    else: return 0

In [ ]:
# Function for counting Personal pronouns
def personal_counter(sent):
    personal_list = ['I','we','We','My','my','Ours','ours','us']
    count = 0
    for i in personal_list:
        if i in sent:
            count += 1
    return count

In [ ]:
# Converting input file into a Dataframe
dataset = pd.read_excel('Input.xlsx')

In [ ]:
# Defining Header variable
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Cafari/537.36'}

In [ ]:
# Tokenizing the Stop Words

directory = 'stop_words'
stop_file = ""
for i in os.listdir(directory):
    f = os.path.join(directory,i)
    stop_file += open(f).read()
stop_words_list = nltk.word_tokenize(stop_file.lower())
nltk_stopwords = stopwords.words('english')
tokenizer = RegexpTokenizer(r'\w+')

In [ ]:
# Tokenizing the Postive & Negative Dictionary
negative_dict = open("negative-words.txt").read()
token_negative_dict = nltk.word_tokenize(negative_dict)
positive_dict = open("positive-words.txt").read()
token_positive_dict = nltk.word_tokenize(positive_dict)

In [ ]:
#Creating Lists to save output values
Positive_Score = []
Negative_Score = []
Polarity_Score = []
Subjectivity_Score= []
avg_sentence_length = []
avg_num_word_per_sentence = []
percent_of_complex_words = []
fog_index = []
complex_word_count = []
word_count = []
syllable_per_word = []
personal_pronouns = []
avg_word_length = []

In [ ]:
# Initializing the loop for Scrapping data through the Urls given in the dataset 
for i in range(0,len(dataset)):
    article_texts = ""
    file_name = int(dataset['URL_ID'].iloc[i])
    url = dataset['URL'].iloc[i]
    try:
        result = requests.get(url,headers = headers)
    except Exception as e:
        print(e)
    soup = bs(result.content,'html.parser')

    #Extracting article title
    page_title= soup.title.text

    # Saving Extracted article into Text files
    try:
        with open(str(file_name)+'.txt','a',encoding="utf-8") as f:
            f.write(page_title)
            f.write('\n')
    except Exception as e:
        print(e)
    # Looping through all the Paragraphs in the article
        for el in soup.find_all('p'):
            article_texts += el.get_text()
            f.write(el.get_text())
            f.write('\n')
        f.close()
    tokenized_articles = nltk.word_tokenize(article_texts.lower())
    cleaned_tokenized_article = tokenizer.tokenize(article_texts.lower())
    
    #Filtering the words present in the Stopwords
    tokens_wot_sw = [word for word in tokenized_articles if not word in stop_words_list]
    nltk_tokens_sw = [word for word in cleaned_tokenized_article if not word in nltk_stopwords]
    
    # Initializing Positive and Negative Index Counter
    positive_index = 0
    negative_index= 0
    for j in tokens_wot_sw:
        if j in token_positive_dict:
            positive_index = positive_index+1

        elif j in token_negative_dict:
            negative_index = negative_index +1
    
    Positive_Score.append(positive_index)
    Negative_Score.append(negative_index)
    Polarity_Score.append((positive_index - negative_index)/ ((positive_index + negative_index) + 0.000001))
    Subjectivity_Score.append((negative_index +  positive_index)/ (len(tokens_wot_sw) + 0.000001))
    try:
        avg_sentence_length.append(len(tokenized_articles)/len(sent_tokenize(article_texts)))
    except ZeroDivisionError:
        avg_sentence_length.append(0)
    try:
        syllable_per_word.append([syllable_counter(x) for x in tokenized_articles])
    except Exception as e:
        print(e)
    try:
        complex_word_count.append([complex_word(x) for x in tokenized_articles])
    except Exception as e:
        print(e)
    try:
        percent_of_complex_words.append([k/len(article_texts) for k in complex_word_count[-1]])
    except Exception as e:
        print(e)
    try:
        fog_index.append([0.4*(avg_sentence_length[-1] + k) for k in percent_of_complex_words[-1]])
    except Exception as e:
        print(e)
    try:
        word_count.append(len(cleaned_tokenized_article))
    except Exception as e:
        print(e)
    try:
        personal_pronouns.append(personal_counter(article_texts))
    except Exception as e:
        print(e)
    try:
        avg_num_word_per_sentence.append(len(tokenized_articles)/len(sent_tokenize(article_texts)))
    except ZeroDivisionError:
        avg_num_word_per_sentence.append(0)
    try:
        avg_word_length.append([len(i)/len(tokenized_articles) for i in tokenized_articles])
    except ZeroDivisionError:
        avg_word_length.append(0)

In [ ]:
#Creating a Dataframe
output_df = pd.DataFrame()

In [ ]:
#Mapping the values of the output columns with respective Outputs
output_df = output_df.assign(**{'URL':dataset["URL"],'POSITIVE SCORE':Positive_Score,'NEGATIVE SCORE':Negative_Score,'POLARITY SCORE':Polarity_Score,'SUBJECTIVITY SCORE':Subjectivity_Score,'AVG SENTENCE LENGTH':avg_sentence_length,'PERCENTAGE OF COMPLEX WORDS':percent_of_complex_words,'FOG INDEX':fog_index,'AVG NUMBER OF WORDS PER SENTENCE':avg_num_word_per_sentence,'COMPLEX WORD COUNT':complex_word_count,'WORD COUNT':word_count,'SYLLABLE PER WORD':syllable_per_word,'PERSONAL PRONOUNS':personal_pronouns,'AVG WORD LENGTH':avg_word_length})

In [ ]:
# Saving the Dataframe as a CSV file
output_df.to_csv('output.csv')